In [ ]:
# modulo para controlar o navegador web
from selenium import webdriver

#localizador de elementos
from selenium.webdriver.common.by import By

# serviço para configurar o caminho do executavel chomedriver
from selenium.webdriver.chrome.service import Service

# classe que permite executar ações avançadas( o mover do mouse, clique/arrasta
from selenium.webdriver.common.action_chains import ActionChains

# classe que espera de forma explicita ate que uma condicao seja satisfeita (ex: que um elemento apareça)
from selenium.webdriver.support.ui import WebDriverWait

# condicoes esperadas usadas com webDriveWait
from selenium.webdriver.support import expected_conditions as ec

# trabalha com dataframes
import pandas as pd

# uso de funções relacionados ao tempos
import time 

# definir caminho do chomedriver
from selenium.common.exceptions import TimeoutException

# inicialização do webDriver

chrome_driver_path = r"C:\Users\48434487888\Documents\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# configuracao do webDriver
servico = Service(chrome_driver_path) #navegador controlado pelo selenium
controle = webdriver.ChromeOptions() #configurar opcoes do navegador
controle.add_argument("--disable-gpu") #evita erros graficos
controle.add_argument("--window-size=1920,1080") #define uma resolução simples

#inicialização do webDriver
driver = webdriver.Chrome(service=servico, options=controle)

# URl inicial
url_base ="https://www.kabum.com.br/espaco-gamer/cadeiras-gamer"
driver.get(url_base)
time.sleep(5) #aguarda 5 segundos para garantir que a pagina carregue 


#criar um dicionario para armazenar os nomes e preços dasa cadeiras 
dick_produtos = {"marca":[], "preço":[]}

#inicia uma pagina, pagina 1 e incrementamos a cada troca de pagina 
pagina = 1

while True:
    print(f"\n coletandos dados da pagina{pagina}...")
    try:
        WebDriverWait(driver, 10).until(
            ec.presence_of_all_elements_located((By.CLASS_NAME,"productCard"))
            )
        print("Elementos encontrados com sucesso!")
    except TimeoutException:
        print("Tempo de espera excedido!")
        
    produtos = driver.find_elements(By.CLASS_NAME,"productCard")
    
    for produto in produtos:
        try:
            nome = produto.find_element(By.CLASS_NAME, "nameCard").text.strip()
            preco = produto.find_element(By.CLASS_NAME, "priceCard").text.strip()
            print(f"{nome} - {preco}")
             
            dick_produtos["marca"].append(nome)
            dick_produtos["preço"].append(preco)
            
        except Exception:
            print("Erro ao coletar dados:", Exception)
            
# encontrar botão da proxima pagina
    try:
        # encontrar o elemento 
        botao_proximo = WebDriverWait(driver, 5).until(
            ec.element_to_be_clickable((By.CLASS_NAME, "nextLink"))
        )
        if botao_proximo:
            driver.execute_script("arguments[0].scrollIntoView();", botao_proximo)
            time.sleep(1)
            
            # clicar no botão
            driver.execute_script("arguments[0].click();", botao_proximo)
            print(f"indo para a pagina {pagina}")
            pagina += 1 
            
            time.sleep(5)
        
        else:
            print("você chegou na ultima pagina!")
            break
        
    except Exception as e :
        print("Erro a tentar avançar para a proxima página", e)
        break
    

# fecha o navegador
driver.quit()

df = pd.DataFrame(dick_produtos)
df.to_excel("cadeiras.xlsx", index=False)


print("Arquivo 'cadeiras' salvo com sucesso ({len(df)} produtos capturados!)")
